The purpose of this workbook is to get a cumulative and accurate wire center base for other reporting functions.
The code obtains raw wire center information from sales table and from Matthew's wire center base, then merges them to form a unique/accurate wire center base.

The reason this is broken out into its own workbook is the amount of time the wire center data takes to run, and so to keep the process efficient, the code is run during down time and the resulting output is exported to an excel sheet once per week to be utilized for other reporting.

Note: wire center base should not change that much, but running once a week will assure accuracy in case there are changes

In [1]:
import os
import sys
import pandas as pd
import numpy as np
import cx_Oracle
cx_Oracle.init_oracle_client(lib_dir=r"C:\Oracle\instantclient_21_6")

In [2]:
#CCDWO1P - SALES WC base
conn = cx_Oracle.connect(
    user="AD21236",
    password="C#ntury123",
    dsn="racorap33-scan.corp.intranet:1521/ccdw01p_users",
    encoding="UTF-8")

#crdmprd - WC base from Matt
conn2 = cx_Oracle.connect(
    user="AD21236",
    password="C3ntury123",
    dsn="crdm.qintra.com:1526/crdmprd",
    encoding="UTF-8")

In [16]:
%%time

#qualifications
# 46 mins
query = """

    select  distinct
            WIRE_CENTER_ID
            ,STATE
            ,CBSA_NAME
            ,DMA_L2

    from 
            rpcarte.dd_t5846_hh_fp_sbg t
 
    where   1=1
            and ilec_ops = 'LUMEN'
 

"""

df_ora = pd.read_sql(query, con=conn)

C:\Users\AD21236\Anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


CPU times: total: 0 ns
Wall time: 34min 51s


In [17]:
# raw data - cleaning after to group unique wire center id and concatenate other fields if applicable
df_ora.to_excel("sales_db_wire_center_mapping.xlsx")

In [18]:
%%time

wc_query = """

            select  DISTINCT 
                    COALESCE(WC_CBSA.CLLI, WC_DMA.WC) AS WIRE_CENTER_ID
                    ,WC_CBSA.CBSA_NAME
                    ,WC_DMA.DMA_NAME
                    
            FROM    CSDAT.WC_DOM_CBSA WC_CBSA
                    FULL JOIN CSDAT.WC_DOM_DMA WC_DMA
                        ON WC_CBSA.CLLI = WC_DMA.WC
                    LEFT JOIN geomkt.brspd_wire_center brightspeed
                        ON COALESCE(WC_CBSA.CLLI, WC_DMA.WC) = brightspeed.WIRE_CENTER_ID
                        
            WHERE   brightspeed.WIRE_CENTER_ID is null --this should filter out all brightspeed wire centers

"""

wc_ora = pd.read_sql(wc_query, con=conn2)

C:\Users\AD21236\Anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


CPU times: total: 31.2 ms
Wall time: 6.24 s


In [19]:
wc_ora.shape

(1739, 3)

In [20]:
# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)
# pd.set_option('display.width', None)
# pd.set_option('display.max_colwidth', None)

In [21]:
wc_ora.head()

,WIRE_CENTER_ID,CBSA_NAME,DMA_NAME
0,ABRDSDCO,"ABERDEEN, SD",SIOUX FALLS (MITCHELL) SD
1,ABRDWA01,"ABERDEEN, WA",SEATTLE-TACOMA WA
2,ACKLIACO,NON-CBSA,DES MOINES-AMES IA
3,ADAROR21,"CORVALLIS, OR",EUGENE OR
4,ADELIACO,"DES MOINES-WEST DES MOINES, IA",DES MOINES-AMES IA


In [22]:
# cleaning data to get state column

wc_ora["DMA_NAME"] = wc_ora["DMA_NAME"].fillna(' ')
wc_ora['STATE1'] = wc_ora['CBSA_NAME'].str.rsplit(',').str[-1] 
wc_ora['STATE2'] = [x.rsplit(" ", 1)[-1] for x in wc_ora["DMA_NAME"]]
wc_ora['STATE2'] = wc_ora['STATE2'].str.strip('()')
wc_ora['STATE'] = np.where(wc_ora['STATE1'] == 'NON-CBSA',wc_ora['STATE2'],wc_ora['STATE1'])
wc_ora['STATE'] = wc_ora['STATE'].str.replace(' ', '')

In [23]:
wc_ora = wc_ora[['WIRE_CENTER_ID','STATE','CBSA_NAME','DMA_NAME']]

In [24]:
wc_ora.shape

(1739, 4)

In [25]:
# check state values
wc_ora.STATE.value_counts()

CO          207
WA          173
AZ          130
FL          125
OR          123
MN          111
IA          106
MN-WI        97
MT           74
NM           70
UT           63
ID           60
NE           50
SD           43
WI           36
OR-WA        29
NE-IA        24
ND           24
NV           21
ND-MN        18
WY           16
IA-NE-SD     12
IA-IL         7
UT-ID         7
OR-ID         6
GA            5
              4
AL            2
IA-IL-MO      2
ID-WA         2
IL            1
WI-MN         1
WY-ID         1
TX            1
Name: STATE, dtype: int64

In [26]:
# Import sales dc wc info
Sales_WC_excel = pd.read_excel(r'sales_db_wire_center_mapping.xlsx',sheet_name = 'Sheet1')

In [27]:
Sales_WC_excel.head()

,WIRE_CENTER_ID,STATE,CBSA_NAME,DMA_L2
0,ESMNWI,WI,NON-CBSA,LA CROSSE-EAU CLAIRE
1,WHTNNDBC,MN,"WAHPETON, ND-MN",FARGO-VALLEY CITY
2,WNDMMNWI,MN,NON-CBSA,MINNEAPOLIS-ST. PAUL
3,DNBRIACO,IA,"SIOUX CITY, IA-NE-SD",SIOUX CITY
4,STMCMN,MN,"MINNEAPOLIS-ST. PAUL-BLOOMINGTON, MN-WI",MINNEAPOLIS-ST. PAUL


In [28]:
Sales_WC_excel['WIRE_CENTER_ID'].nunique()

1879

In [29]:
Sales_final_wc_list = Sales_WC_excel.groupby('WIRE_CENTER_ID')['STATE','CBSA_NAME','DMA_L2'].agg(lambda x: ','.join(x.dropna())).reset_index()

C:\Windows\Temp\ipykernel_16464\1500374550.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  Sales_final_wc_list = Sales_WC_excel.groupby('WIRE_CENTER_ID')['STATE','CBSA_NAME','DMA_L2'].agg(lambda x: ','.join(x.dropna())).reset_index()


In [30]:
# combine wc's from multiple sources
wc_combined = wc_ora.merge(Sales_final_wc_list, how = 'outer', on = 'WIRE_CENTER_ID')

In [31]:
wc_combined.head()

,WIRE_CENTER_ID,STATE_x,CBSA_NAME_x,DMA_NAME,STATE_y,CBSA_NAME_y,DMA_L2
0,ABRDSDCO,SD,"ABERDEEN, SD",SIOUX FALLS (MITCHELL) SD,SD,"ABERDEEN, SD",SIOUX FLS(MCHL)
1,ABRDWA01,WA,"ABERDEEN, WA",SEATTLE-TACOMA WA,WA,"ABERDEEN, WA",SEATTLE-TACOMA
2,ACKLIACO,IA,NON-CBSA,DES MOINES-AMES IA,IA,NON-CBSA,DES MOINES-AMES
3,ADAROR21,OR,"CORVALLIS, OR",EUGENE OR,OR,"CORVALLIS, OR",EUGENE
4,ADELIACO,IA,"DES MOINES-WEST DES MOINES, IA",DES MOINES-AMES IA,IA,"DES MOINES-WEST DES MOINES, IA",DES MOINES-AMES


In [32]:
# Getting non-null state/cbsa/dma final columns

wc_combined['STATE'] = np.where(wc_combined['STATE_x'].isna(),wc_combined['STATE_y'],wc_combined['STATE_x'])
wc_combined['CBSA_NAME'] = np.where(wc_combined['CBSA_NAME_x'].isna(),wc_combined['CBSA_NAME_y'],wc_combined['CBSA_NAME_x'])
wc_combined['DMA_NAME'] = np.where(wc_combined['DMA_NAME'].isna(),wc_combined['DMA_L2'],wc_combined['DMA_NAME'])

#drop other columns
wc_combined = wc_combined.drop(['STATE_x','CBSA_NAME_x','STATE_y','CBSA_NAME_y','DMA_L2'], axis = 1)

In [33]:
wc_combined['STATE2'] = [x.rsplit(" ", 1)[-1] for x in wc_combined["DMA_NAME"]]

In [34]:
wc_combined['STATE'] = np.where(wc_combined['STATE'].isna(),wc_combined['STATE2'],wc_combined['STATE'])
wc_combined = wc_combined.drop(['STATE2'], axis = 1)

In [35]:
#rearrange columns

wc_combined = wc_combined[['WIRE_CENTER_ID', 'STATE', 'CBSA_NAME', 'DMA_NAME']]

In [36]:
#check data
wc_combined.shape # 1967 rows
wc_combined['WIRE_CENTER_ID'].nunique() #1967 unique wire centers

1967

In [37]:
# Import local wc list
local_wc_copy = pd.read_excel(r'Scott_WC_list.xlsx',sheet_name = 'Sheet1')

In [38]:
#clean field names
local_wc_copy = local_wc_copy.rename(columns = {'Primary\nWirecenter ID ': 'WIRE_CENTER_ID',
                                                'CBSA ': 'CBSA',
                                                'DMA ': 'DMA'})

In [39]:
wc_final = wc_combined.merge(local_wc_copy, how = 'outer', on = 'WIRE_CENTER_ID')

In [40]:
# wc_final.shape #3744
# wc_final['WIRE_CENTER_ID'].nunique() #3744
wc_final.head()

,WIRE_CENTER_ID,STATE,CBSA_NAME,DMA_NAME,State,DMA,CBSA
0,ABRDSDCO,SD,"ABERDEEN, SD",SIOUX FALLS (MITCHELL) SD,SD,SIOUX FALLS (MITCHELL) SD,"ABERDEEN, SD"
1,ABRDWA01,WA,"ABERDEEN, WA",SEATTLE-TACOMA WA,WA,SEATTLE-TACOMA WA,"ABERDEEN, WA"
2,ACKLIACO,IA,NON-CBSA,DES MOINES-AMES IA,IA,DES MOINES-AMES IA,NON-CBSA
3,ADAROR21,OR,"CORVALLIS, OR",EUGENE OR,OR,EUGENE OR,"CORVALLIS, OR"
4,ADELIACO,IA,"DES MOINES-WEST DES MOINES, IA",DES MOINES-AMES IA,IA,DES MOINES-AMES IA,"DES MOINES-WEST DES MOINES, IA"


In [41]:
# merge columns

wc_final['STATE'] = np.where(wc_final['STATE'].isna(),wc_final['State'],wc_final['STATE'])
wc_final['CBSA_NAME'] = np.where(wc_final['CBSA_NAME'].isna(),wc_final['CBSA'],wc_final['CBSA_NAME'])
wc_final['DMA_NAME'] = np.where(wc_final['DMA_NAME'].isna(),wc_final['DMA'],wc_final['DMA_NAME'])
wc_final = wc_final.drop(['State','DMA','CBSA'], axis = 1)

In [42]:
# wc_final.shape #3744
# wc_final['WIRE_CENTER_ID'].nunique() #3744
wc_final.head()

,WIRE_CENTER_ID,STATE,CBSA_NAME,DMA_NAME
0,ABRDSDCO,SD,"ABERDEEN, SD",SIOUX FALLS (MITCHELL) SD
1,ABRDWA01,WA,"ABERDEEN, WA",SEATTLE-TACOMA WA
2,ACKLIACO,IA,NON-CBSA,DES MOINES-AMES IA
3,ADAROR21,OR,"CORVALLIS, OR",EUGENE OR
4,ADELIACO,IA,"DES MOINES-WEST DES MOINES, IA",DES MOINES-AMES IA


In [43]:
wc_final.to_excel("FINAL_wire_center_mapping_DO-NOT-ALTER.xlsx", index = False)

# conn.close()

In [36]:
#after export, manually check and fill in states